## **Линейная регрессия**

### **Матричные Операции**

Существует несколько вариантов для выбора **гипераметров** линейной регрессии посмотрим
- Через прямые матричные операции (посмотрим здесь)
- Через градиентны спуск


### **Предсказание цены жилья**

- Загружаем датасет про цены домов в Бостоне; нужно предсказать медиану цены жилья в 506 районах
- Это задача регрессии (предсказываем непрерывные числа)
- Для оценки модели будем использовать RMSE, MSE
- Проверим обобщающую способность модели с помощью train_test_split

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
# работаем с тензорами без req_grad
import torch

y = df['medv']
X = df.drop(['medv'],axis=1)

# Добавляем вектор из единиц к нашим данным
X = np.hstack([np.ones(X.shape[0])[:,None], X])

# Преобразуем в тензора
X = torch.tensor(X)
y = torch.tensor(y.values)

### **Подход подбора параметров модели**

Линейная регрессия выражается следующей зависимостью:
$$y=X\theta+\epsilon,$$
где
- $X$ — матрица объекты-признаки
- $y$ — вектор целевых значений,

соответствующих:
- $X$, $\theta$ — параметр линейной регрессии, $\epsilon$ — некоторый шум

Из данного следует выражение для $\theta$ как:
$$X^Ty=X^TX\theta \rightarrow \theta=(X^TX)^{-1}X^Ty$$

Реализуем выражение для $\theta$ с помощью операций линейной алгебры:

In [6]:
from torch.linalg import inv
from torch import matmul

# функция для вычисления параметров модели
def linreg_linear(X,y):
    lsm = inv(matmul(X.transpose(1,0),X))
    Xt = matmul(X.transpose(1,0),y.double())
    theta = matmul(lsm,Xt)
    return theta

# Получаем параметры модели
theta = linreg_linear(X,y)

# Предсказание модели на данных
y_pred = matmul(X,theta)
y_pred[:10]


tensor([30.0038, 25.0256, 30.5676, 28.6070, 27.9435, 25.2563, 23.0018, 19.5360,
        11.5236, 18.9203], dtype=torch.float64)

Посчитать значение ошибку RMSE для тренировочных данных

In [7]:
# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

import torch
from torch import nn

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [9]:
from torch.nn import MSELoss
import torch

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)

21.8948311817292 4.679191402553351


#### **Проверим обобщающую способность модели**

- Мы обучили модель на некой выборке, и на них же посчитали метрику качества
- Мы не проверили обобщающую способность модели
- Разбиваем выборку на train/valid, вычисляем theta
- Делаем предсказание и вычисляем ошибки

In [10]:
from sklearn.model_selection import train_test_split

# Разделяем выборку та тестовую и валидационную часть
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5)

# вычисляем theta на train
theta = linreg_linear(X_train, y_train)

# Предсказание у нас через matmul
y_pred_tr = matmul(X_train,theta)
y_pred_v = matmul(X_valid,theta)


In [11]:
# Оцениваем качество модели
print_regression_metrics(y_train,y_pred_tr)
print_regression_metrics(y_valid,y_pred_v)

22.29916116189361 4.722198869371515
25.11109360462089 5.011097145797604


В sklearn есть реализация подобного подхода (для удобства)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_valid)
mean_squared_error(y_pred,y_valid)

25.111093604623424

### **Заключение**

В этом ноутбуке мы рассмотрели как можно с помощью матречных операции создать модель линейной регрессии, сделали мы это на примере задачи регрессии, в которой мы предсказали цены на жильё